In [1]:
from dotenv import load_dotenv
import os

from typing import TypedDict, Annotated
from langgraph.checkpoint.memory import MemorySaver
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage, SystemMessage, ToolMessage
from langgraph.prebuilt import ToolNode, InjectedState
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.types import Command
from langchain_core.tools import tool, InjectedToolCallId


from gen_test_cases import generate_test_cases_from_requirements
from gen_requirements import generate_requirements_from_doc 

load_dotenv()
llm_api_key = os.getenv("GOOGLE_API_KEY")

In [2]:

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens = None,
    api_key = llm_api_key,
)

In [3]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    project_name: str
    context: str
    requirements: list
    testCases: list


In [ ]:
@tool
def generate_requirements_from_document_pdf_tool(
    x: Annotated[str, "the PATH to the pdf document from which the tool will generate requirements"],
    tool_call_id: Annotated[str, InjectedToolCallId]
) -> Command:
    """
    Extracts a brief summary and a list of requirements from a project document.

    **Inputs:**
    - `x` (str): The PATH to the pdf document
    
    **Behavior:**
    - Decodes the PDF document from input path into base64 string.
    - Analyzes the document content to extract project requirements.
    - Generates a summary of the document.

    **Outputs:**
    - Updates `state["context"]` with the extracted summary.
    - Updates `state["requirements"]` with a structured list of requirements.
    - Adds a success message in `state["messages"]`.

    **Error Handling:**
    - If the PDF document is invalid or cannot be processed, returns an error message.
    """
    requirements, context = generate_requirements_from_doc(x)
    return Command(update={
        "context": context,
        "requirements": requirements,
        "messages": [
            ToolMessage(
                "Successfully created requirements from the document",
                tool_call_id=tool_call_id
            )
        ]
    })


In [5]:
@tool
def generate_testCases_fromRequirements_tool(
    state: Annotated[AgentState, InjectedState],
    tool_call_id: Annotated[str, InjectedToolCallId]
) -> Command:
    """
    Generates a list of test cases from provided requirements.

    **Inputs:**
    - `state["requirements"]` (list[str]): A list of software or system requirements.
    - `state["context"]` (str): A brief summary to provide additional context.

    **Behavior:**
    - If `requirements` or `context` are missing, returns an error message.
    - Processes requirements to generate structured test cases.
    - Returns a list of test cases in `state["testCases"]`.

    **Outputs:**
    - Updates `state["testCases"]` with generated test cases.
    - Adds a success message to `state["messages"]`.
    """
    requirements = state.get('requirements')
    context = state.get('context')
    if not requirements or not context:
        return Command(update={
            "messages": [
            ToolMessage(
                "Requirements or Context missing",
                tool_call_id=tool_call_id
            )
        ]
    })
    testCases_list = generate_test_cases_from_requirements(requirements, context)
    return Command(update={
        "testCases": testCases_list,
        "messages": [
            ToolMessage(
                "Successfully created test cases from the requirements",
                tool_call_id=tool_call_id
            )
        ]
    })

In [6]:

tools = [generate_testCases_fromRequirements_tool,
         generate_requirements_from_document_pdf_tool]
llm_with_tools = llm.bind_tools(tools)
tools_node = ToolNode(tools)

In [7]:

def call_model(state: AgentState):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": response}

In [8]:
def custom_tool_condition(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    if isinstance(last_message, AIMessage) and last_message.tool_calls:
        return "tools_node"
    return END

In [9]:
builder = StateGraph(AgentState)
builder.add_node("call_model", call_model)
builder.add_node("tools_node", tools_node)

builder.add_edge(START,"call_model")
builder.add_conditional_edges(
    "call_model",
    custom_tool_condition,
)
builder.add_edge("tools_node","call_model")
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [10]:
#Initialize state by loading a project and its info (if any)
with open("..\Prompts\main_system_prompt.txt","r") as f:
    main_system_prompt = f.read()
currState = AgentState(messages=[SystemMessage(content=main_system_prompt)],
                       project_name="QA_automation")

<>:2: SyntaxWarning: invalid escape sequence '\P'
<>:2: SyntaxWarning: invalid escape sequence '\P'
C:\Users\HOME\AppData\Local\Temp\ipykernel_11436\3511123684.py:2: SyntaxWarning: invalid escape sequence '\P'
  with open("..\Prompts\main_system_prompt.txt","r") as f:


In [11]:
pdf_path = '../ReqView-Example_Software_Requirements_Specification_SRS_Document-3-5.pdf'
tempt_requirements, tempt_summary = generate_requirements_from_doc(pdf_path)

content=[{'type': 'text', 'text': 'You are a Quality Assurance (QA) professional in the Software Development Life Cycle (SDLC). You are tasked with generating comprehensive requirements list based on a given project document. This is a critical step in ensuring thorough testing coverage for the project. Your goal is to create exhaustive requirements list that cover all aspects of the project.\nHere is the project document you will be working with:'}, {'type': 'document', 'source': {'type': 'base64', 'media_type': 'application/pdf', 'data': '...'}, 'cache_control': {'type': 'ephemeral'}}, {'type': 'text', 'text': 'Now, follow these steps to extract and compile the requirements:\n\n1. Thoroughly analyze the document, paying close attention to explicit and implicit requirements. Look for statements that describe system functionality, user interactions, performance expectations, security measures, and any other aspects of the project.\n\n2. Categorize the requirements into appropriate grou

In [15]:
tempt_requirements[1]['category']

'User Interface Requirements'

In [21]:
# Get user Input

pdf_path = '../ReqView-Example_Software_Requirements_Specification_SRS_Document-3-5.pdf'

userInputContent = pdf_path

# Actually run
currState['messages'].append(HumanMessage(content=userInputContent))
config = {"configurable": {"thread_id": "1"}}
graph.invoke(currState,config)


{'messages': [SystemMessage(content='You are a dedicated QA testing assistant working throughout the Software Development Life Cycle. Your role is to help a QA tester by processing project documents and transforming them into actionable insights in three sequential steps:\n1.Document â†’ Requirements: Extract project requirements from a PDF document.\n2.Requirements â†’ Test Cases: Generate test cases based on the extracted requirements.\n\nYour state is maintained as a Python dictionary with the following attributes:\n- messages: A list of communications (including success messages, error messages, and tool call logs).\n- project_name: The name of the project.\n- context: The document summary extracted from the PDF. (Always use "context" for the summary, not "summary.")\n- requirements: A list of requirements extracted from the project document.\n- testCases: A list of test cases generated from the requirements.\n\nYou have access to two primary tools:\n\n1. PDF Requirements Extractio

In [36]:
graph.get_state({"configurable": {"thread_id": "3"}}).values.get("context")

In [24]:
with open('../ReqView-Example_Software_Requirements_Specification_SRS_Document-1-6.pdf',"rb") as f:
    pdf_data = base64.b64encode(f.read()).decode('utf-8')

requirements1, context1 = generate_requirement_from_doc(pdf_data)

APIConnectionError: Connection error.